In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
78938,Мозамбик,2021-06-15,71651,69937,842,83,48,1
78939,Украина,2020-12-31,1086997,736611,19281,10117,12468,223
78940,Кипр,2020-06-23,990,824,19,2,0,0
78941,Эсватини,2021-01-26,14830,9542,517,208,148,24
78942,Турция,2021-02-19,2624019,2511548,27903,7419,7498,82
78943,Аргентина,2021-03-24,2269877,2049035,54946,8300,5078,123
78944,Алжир,2020-10-01,51690,36282,1741,160,108,5
78945,Бенин,2021-03-13,6501,5552,81,0,0,0
78946,Камбоджа,2020-08-01,240,196,0,1,32,0
78947,Гвинея-Бисау,2020-09-06,2245,1127,38,0,0,0


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
397,Весь мир,2021-07-03,183459631,120569294,3970780,375056,269775,6842
398,Весь мир,2021-07-04,183786486,120868182,3977169,326855,298888,6389
399,Весь мир,2021-07-05,184158258,121235175,3984631,371772,366993,7462
400,Весь мир,2021-07-06,184612003,121563018,3993196,453745,327843,8565
401,Весь мир,2021-07-07,185075852,121831181,4001760,463849,268163,8564
402,Весь мир,2021-07-08,185557387,122123832,4010605,481535,292651,8845
403,Весь мир,2021-07-09,186065732,122462950,4019052,508345,339118,8447
404,Весь мир,2021-07-10,186471199,122878183,4025094,405467,415233,6042
405,Весь мир,2021-07-11,186816068,123142309,4031775,344869,264126,6681
406,Весь мир,2021-07-12,187238474,123428255,4038886,422406,285946,7111


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2021-07-12


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Marshall Islands'), (1, 'Micronesia'), (2, 'Summer Olympics 2020'), (3, 'Австралия'), (4, 'Австрия'), (5, 'Азербайджан'), (6, 'Албания'), (7, 'Алжир'), (8, 'Ангола'), (9, 'Андорра'), (10, 'Антигуа и Барбуда'), (11, 'Аргентина'), (12, 'Армения'), (13, 'Афганистан'), (14, 'Багамские острова'), (15, 'Бангладеш'), (16, 'Барбадос'), (17, 'Бахрейн'), (18, 'Беларусь'), (19, 'Белиз'), (20, 'Бельгия'), (21, 'Бенин'), (22, 'Болгария'), (23, 'Боливия'), (24, 'Босния и Герцеговина'), (25, 'Ботсвана'), (26, 'Бразилия'), (27, 'Бруней'), (28, 'Буркина-Фасо'), (29, 'Бурунди'), (30, 'Бутан'), (31, 'Вануату'), (32, 'Великобритания'), (33, 'Венгрия'), (34, 'Венесуэла'), (35, 'Весь мир'), (36, 'Восточный Тимор'), (37, 'Вьетнам'), (38, 'Габон'), (39, 'Гаити'), (40, 'Гайана'), (41, 'Гамбия'), (42, 'Гана'), (43, 'Гватемала'), (44, 'Гвинея'), (45, 'Гвинея-Бисау'), (46, 'Германия'), (47, 'Гондурас'), (48, 'Гренада'), (49, 'Греция'), (50, 'Грузия'), (51, 'ДР Конго'), (52, 'Дания'), (53, 'Джибут

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
79360,2021-07-08,814319,779855,14865,2230,1519,17,1.038,1.008,0,0,Япония,19599,1754.428571,1633.357143,19019.285714,19286.071429,50,694
79361,2021-07-09,816593,781457,14885,2274,1602,20,1.354,1.009,0,0,Япония,20251,1825.857143,1672.428571,19172.142857,19262.571429,44,652
79362,2021-07-10,819051,783273,14896,2458,1816,11,1.499,1.010,0,0,Япония,20882,1907.857143,1731.857143,19401.000000,19317.928571,184,631
79363,2021-07-11,821083,783291,14902,2032,18,6,1.407,1.010,0,0,Япония,22890,1986.142857,1786.142857,19947.714286,19545.785714,-426,2008
79364,2021-07-12,822594,786425,14905,1511,3134,3,1.159,1.011,0,0,Япония,21264,2053.428571,1822.142857,20339.428571,19692.000000,-521,-1626


In [6]:
# Добавляем колонки расчитанные ,доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


            Confirmed  Remaining_ill
Date                                
2021-07-11  186816068       59641984
2021-07-12  187238474       59771333
79365
(79365, 19)
2020-06-01


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
54518,2021-06-23,5306069,Россия,2.954455,0.570544
54519,2021-06-24,5325940,Россия,2.958866,0.579895
54520,2021-06-25,5346005,Россия,2.963070,0.588712
54521,2021-06-26,5367317,Россия,2.968912,0.599328
54522,2021-06-27,5387486,Россия,2.974969,0.610781
54523,2021-06-28,5408744,Россия,2.981243,0.624589
54524,2021-06-29,5428961,Россия,2.986064,0.632490
54525,2021-06-30,5449594,Россия,2.990957,0.637976
54526,2021-07-01,5472722,Россия,2.996430,0.647817
54527,2021-07-02,5495513,Россия,3.001625,0.653354


In [8]:
X.to_csv('W.csv',index=False)